# Other Maybrain Functionalities

Maybrain also offers some other functionalities that you might find useful.

Let's start by initialising a brainObj:

In [1]:
from maybrain import brainObj as mbt
a = mbt.brainObj()
a.importAdjFile("data/mean_500.txt", delimiter=",")
a.importSpatialInfo("data/parcel_500.txt")
a.applyThreshold()

## Updating the Adjacency Matrix

After working directly with the *networkX* `G` object which you have inside *brainObj*, you can directly update your original adjacency matrix, by getting the values from `G`.

One way is to choose a specific edge to update:

In [2]:
print("Old value at edge (2,2): ", a.adjMat[2][2])
#Updating G with dump edge
a.G.add_edge(2,2, weight=5)
a.updateAdjMat((2,2))
print("New value at edge (2,2): ", a.adjMat[2][2])


Old value at edge (2,2):  0.0
New value at edge (2,2):  5.0


You can also update the whole adjacency matrix at once. Note that in our `G` object there is no edge between a node to itself thus, when reconstructing the adjacency matrix, that value will be `nan`:

In [3]:
print("Weight of edge (0,0) in adjMat: ", a.adjMat[0][0])
a.reconstructAdjMat()
print("Weight of edge (0,0) in adjMat after reconstructing: ", a.adjMat[0][0])

Weight of edge (0,0) in adjMat:  0.0
Weight of edge (0,0) in adjMat after reconstructing:  nan


## Removing Unconnected Nodes

After a while you might have some nodes in `G` which are not connected to any other node. You can remove those nodes:

In [4]:
# Import adjacency file again because adjMat was altered before
a.importAdjFile("data/mean_500.txt", delimiter=",")
a.applyThreshold(thresholdType="totalEdges", value = 1)

print("Number of nodes: ", a.G.number_of_nodes())
a.removeUnconnectedNodes()
print("Number of nodes after remove: ", a.G.number_of_nodes())

Number of nodes:  500
Number of nodes after remove:  2


## Information about Percentages

Imagine that you want to know what would be the ratio between the edges on `adjMat` above a certain threshold value and the total possible edges of `adjMat` (the ones different from `NaN`). This might be useful for you to decide which threshold you might apply later.

In our specific matrix, we can verify that we have 124750 possible edges in `adjMat`, and if we applied a threshold of 0.6, we would get 3387 edges:

In [5]:
a.importAdjFile("data/mean_500.txt", delimiter=",")
print("Ratio:", a.thresholdToPercentage(0.6))

## Checking the previous result
# Creating all possible edges
a.applyThreshold() 
print("Total possible edges: ", a.G.number_of_edges())
# Getting the edges thresholded with 0.6
a.applyThreshold(thresholdType="tVal", value=0.6)
print("Number of edges from a threshold of 0.6: ", a.G.number_of_edges())

print("(3387/124750 = ", 3387/124750, ")")        

Ratio: 0.027150300601202406
Total possible edges:  124750
Number of edges from a threshold of 0.6:  3387
(3387/124750 =  0.027150300601202406 )


While `thresholdToPercentage()` is based on values from `adjMat`, we also have another method to calculate a similar ratio from values of the `G` object. This method is `percentConnected()`, and it returns the ratio of the current number of edges in our `G` object and the total number of possible connections. 

You can see this difference with other aspects. For example, if `adjMat` has *NaN*s, they are not counted in the result of `thresholdToPercentage()`. On the other hand, `percentConnected()` calculates the number of total possible connections, for an unidrected graph, using the following formula:
$$\left (nodes \times \left (nodes - 1  \right )  \right ) / 2$$
This is equivalent to the upper right side of an adjacency matrix, including possible *NaN*s that it might have. 

In [6]:
a.applyThreshold()
print("Ratio: ", a.percentConnected())

Ratio:  1.0


The previous ratio is equals to 1 because we applied a threshold where we included all the possible edges from `adjMat`, thus everything is connected.

We can reach the same ratio value we showed before with `thresholdToPercentage()` if we apply a threshold of 0.6:

In [7]:
a.applyThreshold(thresholdType="tVal", value=0.6)

print("Ratio in a.G after thresholding of 0.6: ", a.percentConnected())

## Checking the previous value
nodes = a.G.number_of_nodes()
print("Total possible edges from G: ", (nodes * (nodes-1)) / 2)
print("Number of edges in G: ", a.G.number_of_edges())

print("(3387/124750 = ", 3387/124750, ")")  

Ratio in a.G after thresholding of 0.6:  0.027150300601202406
Total possible edges from G:  124750.0
Number of edges in G:  3387
(3387/124750 =  0.027150300601202406 )


## Linked Nodes

Looking to the edges of your `G` object you can see which nodes are connected, but you might prefer to have that information in the nodes themselves. 

With `findLinkedNodes()` you can add a property to each node saying to which other nodes it is connected. You can access that property through `a.LINKED_NODES`.

In [8]:
a.applyThreshold(thresholdType="totalEdges", value="5")
print("Our edges are: ", a.G.edges())

a.findLinkedNodes()

#As the graph is undirected, this property works on both directions:
print("Node 341 is linked to: ", a.G.node[341][a.LINKED_NODES])
print("Node 92 is linked to: ", a.G.node[92][a.LINKED_NODES])
print("Node 1 is linked to: ", a.G.node[1][a.LINKED_NODES], " (nothing)")

Our edges are:  [(92, 302), (96, 194), (223, 365), (238, 341), (241, 341)]
Node 341 is linked to:  [238, 241]
Node 92 is linked to:  [302]
Node 1 is linked to:  []  (nothing)


## Weights to Distances

With a weight, the higher the value the stronger the connection. With a distance, the higher the value the "weaker" the connection.

If you want to convert the weights of our edges in a way that they can be seen as distances, you just have to use `weightToDistance()`. Note that there is no measurement unit for the distance, as it just a conversion from the weights. 

Essentially, the value of the maximum weight will be approximately the value of the highest possible distance (if your weights are all positive in the first place). 

In [9]:
print("Our edges before conversion: ")
for e in a.G.edges(data=True):
    print(e)

a.weightToDistance()

print("Our edges after conversion: ")
for e in a.G.edges(data=True):
    print(e)

Our edges before conversion: 
(92, 302, {'weight': 1.2896769679245286})
(96, 194, {'weight': 1.2930219528301885})
(223, 365, {'weight': 1.2869634132075474})
(238, 341, {'weight': 1.2818323018867921})
(241, 341, {'weight': 1.2753403320754713})
Our edges after conversion: 
(92, 302, {'weight': 1.2896769679245286, 'distance': 0.005344984905659933})
(96, 194, {'weight': 1.2930219528301885, 'distance': 0.0020000000000000018})
(223, 365, {'weight': 1.2869634132075474, 'distance': 0.0080585396226411365})
(238, 341, {'weight': 1.2818323018867921, 'distance': 0.013189650943396414})
(241, 341, {'weight': 1.2753403320754713, 'distance': 0.019681620754717244})


You can access weights and distances of specific edges through `a.WEIGHT` and `a.DISTANCE`, respectively:

In [10]:
print("Weight of edge (92,302):   ", a.G.edge[92][302][a.WEIGHT])
print("Distance of edge (92,302): ", a.G.edge[92][302][a.DISTANCE])

Weight of edge (92,302):    1.28967696792
Distance of edge (92,302):  0.00534498490566


## Isosurfaces and Parcels